# Adaptive Large Neighbourhood Search (ALNS) for TOPTW-C

## Problem Overview
The **Team Orienteering Problem with Time Windows and Capacity (TOPTW-C)** is a profit-maximizing VRP variant where:
- A fleet of vehicles (max_routes) starts/ends at depot
- Each customer has: profit, demand, service time, and time window [ready_time, due_time]
- Vehicles have homogeneous capacity Q
- **Not all customers must be visited** (unlike standard VRPTW)
- **Objective: Maximize (profit - λ·cost)** while respecting constraints
- Capacity constraint: ∑(demand_i) ≤ Q for each route

## mathematical formulation

The **Team Orienteering Problem with Time Windows and Capacity Constraints (TOPTW-C)** maximizes:


$$
\max \underbrace{\sum_{k \in K} \sum_{i \in V} p_i \cdot y_{ik}}_{\text{Total Profit}}
- \lambda \underbrace{\sum_{k \in K} \sum_{(i,j) \in A} c_{ij} \cdot x_{ijk}}_{\text{Travel Cost}}
$$

**Subject to:**
- **Time windows**: $a_i \leq t_i \leq b_i$ (hard constraints)
- **Capacity**: $\sum_{i \in V} d_i \cdot y_{ik} \leq Q$ per vehicle
- **Max routes**: $|K| \leq K_{\max}$
- **Visit decisions**: $y_{ik} \in \{0,1\}$ (1 if customer i served by vehicle k)

**Big-M Formulation Note**: The constraints use "Big-M" logic to enforce feasibility only when yᵢₖ = 1 (customer visited). The compatibility matrix (Section 2.2) pre-checks these conditions without solving the full NP-hard problem.

**Parameters:**
- $p_i$ = profit, $d_i$ = demand, $[a_i,b_i]$ = time window
- $x_{ijk}$ = 1 if arc (i,j) used by vehicle k
- $y_{ik}$ = visit decision variable (binary)
- $\lambda$ = cost tradeoff parameter (higher λ = prioritize cost savings)

**Key Insight**: The λ parameter controls the profit-cost tradeoff:
- λ = 0: Pure profit maximization (ignore travel costs)
- λ → ∞: Minimize travel distance (visit few high-profit customers)
- Typical λ = 0.01-0.1 balances both objectives

**Depot Convention**: Customer index `0` is always the depot at coordinates (0,0). All routes must start/end here.

## 1. Data Model & Instance Representation

### Design Decisions
- **Customer 0 is depot**: All routes must start/end at index 0
- **Symmetric distances**: Travel time = Euclidean distance
- **TOPTWInstance dataclass**: Bundles all problem data for easy serialization

### Key Assumptions
- Time = distance (can be modified in `travel_time()`)
- Capacity Q is homogeneous across fleet
- Service times are additive to travel time

In [8]:
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import math
import random

@dataclass
class Customer:
    idx: int          # 0 is depot
    x: float
    y: float
    demand: float
    ready_time: float   # earliest start of service e_i
    due_time: float     # latest start of service l_i
    service_time: float # s_i
    profit: float  # ADD THIS

@dataclass
class TOPTWInstance:
    Q: float                         # vehicle capacity
    customers: List[Customer]        # 0 is depot / 1..n are customers
    dist: List[List[float]]          # distance matrix (symmetric here)

    def travel_time(self, i: int, j: int) -> float:
        # assume time == distance
        return self.dist[i][j]

def euclidean(a: Customer, b: Customer) -> float:
    return math.hypot(a.x - b.x, a.y - b.y)

def build_distance_matrix(customers: List[Customer]) -> List[List[float]]:
    n = len(customers)
    D = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            d = euclidean(customers[i], customers[j])
            D[i][j] = D[j][i] = d
    return D


## 2. Instance Loader & Time Compatibility

### 2.1 JSON Parser for TOPTW Instances
Loads custom JSON format with customer data, time windows, profits, and capacity.
Returns TOPTWInstance object with precomputed distance matrix.


In [9]:
import json

def parse_toptw_json(path: str):
    """Parse TOPTW JSON file into (inst, K)"""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    Q = float(data["Q"])
    K = int(data.get("k", 3))  # max routes from your JSON

    customers = []
    for node in data["nodes"]:
        customers.append(Customer(
            idx=int(node["i"]),
            x=float(node["x"]),
            y=float(node["y"]),
            demand=float(node["demand"]),
            ready_time=float(node["tw_open"]),
            due_time=float(node["tw_close"]),
            service_time=float(node["service"]),
            profit=float(node["profit"]),
        ))

    D = build_distance_matrix(customers)
    inst = TOPTWInstance(Q=Q, customers=customers, dist=D, max_routes=K)
    return inst, K

### 2.2 Time Compatibility Precomputation

**Purpose**: Avoid O(n³) insertion checks by precomputing O(n²) feasibility matrix.

**Complexity**:
- Build: $O(n^2)$ once at startup
- Query: $O(1)$ per insertion check
- Speedup: ~10-100x vs full route simulation

**Implementation Note**: Uses "Big-M" style reasoning - checks necessary conditions before sufficient ones.

**Performance Comparison:**

| Check Method | Complexity | When to Use |
|--------------|------------|-------------|
| Full simulation | O(n) per insertion | Final validation only |
| Bounds check | O(1) per insertion | Fast pruning (80% speedup) |

In [10]:
def build_time_compatibility_matrix(inst: TOPTWInstance) -> Dict[Tuple[int, int], bool]:
    """
    Precompute which customer pairs (i,j) are time-compatible.
    Customer j can follow customer i if:
      earliest_arrival_at_j = ready[i] + service[i] + travel[i,j] <= due[j]

    This is inspired by the Big-M formulation where infeasible arcs
    can be detected without full route simulation.
    """
    n = len(inst.customers)
    compatible = {}

    for i in range(n):
        for j in range(n):
            if i == j:
                compatible[(i, j)] = False
                continue

            cust_i = inst.customers[i]
            cust_j = inst.customers[j]

            # Earliest we can arrive at j after serving i
            earliest_arrival = (cust_i.ready_time +
                               cust_i.service_time +
                               inst.dist[i][j])

            # Can we arrive within j's time window?
            # We can wait if early, but cannot be late
            compatible[(i, j)] = (earliest_arrival <= cust_j.due_time)

    return compatible


def compute_insertion_bounds(inst: TOPTWInstance,
                            predecessor: int,
                            successor: int,
                            new_cust: int) -> Tuple[float, float]:
    """
    Compute time bounds for inserting new_cust between predecessor and successor.
    Returns (earliest_feasible, latest_feasible) arrival times at new_cust.

    Uses Big-M-style reasoning:
    - earliest: pushed by predecessor's time window
    - latest: constrained by successor's time window

    If earliest > latest, insertion is infeasible.
    """
    i, k, j = predecessor, new_cust, successor

    cust_i = inst.customers[i]
    cust_k = inst.customers[k]
    cust_j = inst.customers[j]

    # Earliest arrival at k (pushed from i)
    earliest = max(
        cust_i.ready_time + cust_i.service_time + inst.dist[i][k],
        cust_k.ready_time
    )

    # Latest arrival at k (pulled back from j's deadline)
    latest = min(
        cust_j.due_time - cust_k.service_time - inst.dist[k][j],
        cust_k.due_time
    )

    return earliest, latest


def fast_insertion_check(inst: TOPTWInstance,
                         route: List[int],
                         insert_pos: int,
                         new_cust: int) -> bool:
    """
    Fast feasibility check using time bounds before full simulation.
    Returns True if insertion might be feasible.
    """
    if insert_pos >= len(route) - 1:
        return False

    i = route[insert_pos]
    j = route[insert_pos + 1]

    earliest, latest = compute_insertion_bounds(inst, i, j, new_cust)

    # Quick rejection if time bounds are infeasible
    if earliest > latest + 1e-9:
        return False

    # Passed bounds check - still need full validation
    return True

## 3. Data Loading & Evaluation Framework

**Purpose**: Load TOPTW instances from JSON files and define solution evaluation metrics.

This section provides:
1. **File loader**: Walks directories to find and parse TOPTW JSON instances
2. **Evaluation functions**: Calculate profit, cost, and feasibility for TOPTW solutions
3. **Benchmark preparation**: Prepares instances for later experimentation

**Key metrics defined here** (but not yet computed):
- `profit`: Sum of visited customer profits
- `cost`: Total travel distance  
- `net_value`: profit_weight × profit - cost_weight × cost
- `feasible`: Whether solution respects capacity, time windows, and max routes

**Note**: The actual ALNS execution and benchmark comparisons will happen in Sections 8-9, after the algorithm is defined.

In [11]:
def flatten_routes(solution: List[List[int]]) -> List[int]:
    seq = []
    for r in solution:
        seq.extend(r[1:-1])  # skip depot endpoints
    return seq

In [12]:
import os
import pandas as pd
import numpy as np
import time
from typing import Optional  # ADD for compatibility


###############################################################################
# File loader
###############################################################################
def load_all_instances_from_folder(
    folder: str,
    patterns=(".json", ".JSON"),  # ✅ CHANGE 1: Now only JSON
    select: str = "all",
    n: Optional[int] = None,
    random_seed: int = 1234
):
    """
    ✅ CHANGE 2: Updated docstring for TOPTW JSON files
    Walk `folder`, parse TOPTW JSON files, and return a list of (name, inst, K).
    """
    out = []
    for root, _, files in os.walk(folder):
        for fn in files:
            if fn.endswith(patterns):
                p = os.path.join(root, fn)
                try:
                    # ✅ CHANGE 3: Use JSON parser (not Solomon)
                    inst, K = parse_toptw_json(p)
                    name = os.path.splitext(fn)[0]
                    out.append((name, inst))
                except Exception as e:
                    print(f"[WARN] Skipping {fn}: {e}")

    # deterministic order first
    out.sort(key=lambda t: t[0])

    if select == "all" or n is None:
        return out

    if select == "first_n":
        return out[:max(0, min(n, len(out)))]

    if select == "random_n":
        rng = random.Random(random_seed)
        n_eff = max(0, min(n, len(out)))
        return rng.sample(out, n_eff)

    raise ValueError(f"Unknown select mode: {select!r}")




# ============================================================================
# INSERT TOPTW EVALUATION HERE (NEW CODE)
# ============================================================================

def total_profit(inst: TOPTWInstance, solution: List[List[int]]) -> float:
    """Calculate total profit of visited customers"""
    visited = set(flatten_routes(solution))
    return sum(inst.customers[i].profit for i in visited)

def solution_feasible_toptw(inst: TOPTWInstance, solution: List[List[int]]) -> bool:
    """Check feasibility for TOPTW (no requirement to visit all)"""
    seen = set()
    for route in solution:
        if route[0] != 0 or route[-1] != 0:
            return False
        if not route_capacity_ok(inst, route):
            return False
        if not route_time_feasible(inst, route):
            return False
        if len(solution) > inst.max_routes:
            return False
        for c in route[1:-1]:
            if c in seen:
                return False  # duplicate visit
            seen.add(c)
    return True

def evaluate_solution_toptw(inst: TOPTWInstance, sol: list[list[int]]) -> dict:
    """KPIs for TOPTW solution"""
    feasible = solution_feasible_toptw(inst, sol)
    return {
        "feasible": feasible,
        "profit": total_profit(inst, sol) if feasible else 0,
        "cost": total_distance(inst, sol) if feasible else float('inf'),
        "num_routes": len(sol),
        "num_customers": len(flatten_routes(sol)),
    }

###############################################################################
# 8. Final Experiment Runner for TOPTW (Simplified - No MILP Baseline)
###############################################################################

def run_folder_experiment_toptw(
    folder: str,
    repeats: int = 5,
    iters: int = 300,
    base_seed: int = 1000,
    select: str = "all",
    n: Optional[int] = None,      # CHANGE: use Optional
    random_seed: int = 1234,
    profit_weight: float = 1.0,   # ADD
    cost_weight: float = 0.01,    # ADD
):
    """
    Run TOPTW experiments across all JSON instances in a folder.

    For each instance:
      1. Parse TOPTW JSON file
      2. Run ALNS `repeats` times with different seeds
      3. Track profit, cost, net value, feasibility, and runtime
      4. Compare against baseline greedy solution
      5. Return detailed results and summary
    """

    # The load_all_instances_from_folder function now returns (name, inst) tuples,
    # so we don't need to unpack 'K' here.
    instances = load_all_instances_from_folder(
        folder,
        patterns=(".json", ".JSON"),
        select=select,
        n=n,
        random_seed=random_seed,
    )

    rows = []

    # CHANGE: Use inst.max_routes instead of K
    for inst_name, inst in instances:  # Removed '_', assuming K is not needed here
      print(f"=== Running TOPTW: {inst_name} ({len(inst.customers)-1} customers, max_routes={inst.max_routes}) ===")

      # NEW: Compute baseline for comparison (with feasibility check)
      baseline_sol = greedy_initial_solution_toptw(inst, random.Random(0))

      # Check baseline feasibility
      if not solution_feasible_toptw(inst, baseline_sol):
          print(f"Warning: Infeasible baseline for {inst_name}")
          baseline_profit = 0.0
          baseline_cost = float('inf')
          baseline_net = 0.0
      else:
          baseline_profit = total_profit(inst, baseline_sol)
          baseline_cost = total_distance(inst, baseline_sol)
          baseline_net = profit_weight * baseline_profit - cost_weight * baseline_cost

      for r in range(repeats):
        seed = base_seed + r

        # Run ALNS
        t0 = time.time()
        result = alns_enhanced( # Changed from alns to alns_enhanced
            inst,
            iters=iters,
            seed=seed,
            profit_weight=profit_weight,  # PASS parameter
            cost_weight=cost_weight,      # PASS parameter
        )
        t1 = time.time()

        # Evaluate TOPTW solution
        best_sol = result["best_solution"]
        kpis = evaluate_solution_toptw(inst, best_sol)

        # NEW: Compute net value
        net_value = profit_weight * kpis["profit"] - cost_weight * kpis["cost"]

        # Calculate improvement percentage safely (avoid division by zero)
        if abs(baseline_net) > 1e-9:
            improvement_pct = ((net_value - baseline_net) / abs(baseline_net)) * 100
        else:
            improvement_pct = float('inf') if net_value > 0 else 0.0

        # ✅ FIX: Calculate operator usage from iteration logs
        df_log = pd.DataFrame(result["iter_log"])
        op_usage = df_log["op_name"].value_counts(normalize=False).to_dict() if len(df_log) > 0 else {}

        # ENHANCED: Track net metrics and baseline
        rows.append({
            "instance": inst_name,
            "n_customers": len(inst.customers) - 1,
            "max_routes": inst.max_routes,  # Use inst attribute
            "repeat": r,
            "seed": seed,
            "profit": kpis["profit"],
            "improvement_pct": improvement_pct,
            "cost": kpis["cost"],
            "net_value": net_value,
            "baseline_net": baseline_net,
            "num_customers_visited": kpis["num_customers"],
            "num_routes_used": kpis["num_routes"],
            "feasible": kpis["feasible"],
            "time_sec": t1 - t0,
            "final_probs": result["final_probs"],
            "op_usage": op_usage,  # ✅ Now contains real counts
        })

    df_results = pd.DataFrame(rows)

    # Create summary statistics per instance
    grouped = df_results.groupby(["instance", "n_customers", "max_routes"])

    # ENHANCED: Add net value aggregations
    df_summary = grouped.agg(
        runs=("instance", "count"),
        feas_rate=("feasible", "mean"),
        mean_profit=("profit", "mean"),
        std_profit=("profit", "std"),
        max_profit=("profit", "max"),
        mean_net_value=("net_value", "mean"),           # NEW
        max_net_value=("net_value", "max"),             # NEW
        mean_cost=("cost", "mean"),
        mean_time=("time_sec", "mean"),
        avg_customers=("num_customers_visited", "mean"),
        avg_routes=("num_routes_used", "mean"),
        improvement_over_baseline=("improvement_pct", "mean"),  # NEW
    ).reset_index()

    print("\n=== TOPTW SUMMARY BY INSTANCE ===")
    print(df_summary.to_string(index=False))

    return df_results, df_summary

## 4. Feasibility and Cost Helpers

### Solution Representation
We represent a solution as a **list of routes**, where each route includes depot endpoints:
```python
route = [0, 7, 3, 12, 0]  # depot → 7 → 3 → 12 → depot
solution = [route1, route2, ...]  # Multiple vehicle routes

### These functions check if a solution is valid:
- `route_capacity_ok()`: Checks if truck load ≤ capacity
- `route_time_feasible()`: Checks if all time windows are met
- `total_distance()`: Calculates total travel distance

**Why this matters:** ALNS generates thousands of candidate solutions. These fast checks (O(n) each) keep the algorithm speedy.

In [13]:
def route_capacity_ok(inst: TOPTWInstance, route: List[int]) -> bool:
    load = 0.0
    for idx in route:
        if idx == 0:
            continue
        load += inst.customers[idx].demand
        if load > inst.Q + 1e-9:
            return False
    return True

def route_time_feasible(inst: TOPTWInstance, route: List[int]) -> bool:
    t = 0.0  # time clock
    for a, b in zip(route, route[1:]):
        # drive
        t += inst.travel_time(a, b)
        cust_b = inst.customers[b]

        # wait if early
        if t < cust_b.ready_time:
            t = cust_b.ready_time

        # if late -> infeasible
        if t > cust_b.due_time + 1e-9:
            return False

        # serve
        t += cust_b.service_time
    return True

def total_distance(inst: TOPTWInstance, solution: List[List[int]]) -> float:
    dist = 0.0
    for route in solution:
        for a, b in zip(route, route[1:]):
            dist += inst.dist[a][b]
    return dist

In [14]:
# ============================================================================
# Soft Time Windows Support
# ============================================================================

def route_cost_with_soft_tw(inst: TOPTWInstance,
                             route: List[int],
                             cost_weight: float = 1.0,
                             lateness_weight: float = 10.0) -> Tuple[float, float, float]:
    """
    Evaluate route with soft time windows (allows lateness with penalty).

    Returns: (travel_cost, lateness_cost, total_lateness_time)

    Implements the soft-TW constraint from the formulation:
      a_i ≤ t_i ≤ b_i + L_i + M_i(1 - y_i)

    Where y_i = 1 if time window is respected, 0 if late (penalized).
    """
    if len(route) < 3:
        return 0.0, 0.0, 0.0

    t = 0.0
    travel_cost = 0.0
    lateness_cost = 0.0
    total_lateness = 0.0

    for a, b in zip(route, route[1:]):
        # Travel
        travel_time = inst.travel_time(a, b)
        t += travel_time
        travel_cost += inst.dist[a][b] * cost_weight

        cust_b = inst.customers[b]

        # Wait if early
        if t < cust_b.ready_time:
            t = cust_b.ready_time

        # Handle lateness
        if t > cust_b.due_time:
            lateness = t - cust_b.due_time

            # Check if we have soft TW info
            if hasattr(cust_b, 'max_lateness'):
                # Respect maximum lateness
                if lateness > cust_b.max_lateness:
                    return float('inf'), float('inf'), float('inf')

                # Apply penalty
                penalty = lateness * cust_b.lateness_penalty * lateness_weight
                lateness_cost += penalty
                total_lateness += lateness

        # Service
        t += cust_b.service_time

    return travel_cost, lateness_cost, total_lateness


def solution_value_soft_tw(inst: TOPTWInstance,
                           solution: List[List[int]],
                           profit_weight: float = 1.0,
                           cost_weight: float = 0.01,
                           lateness_weight: float = 10.0) -> Dict:
    """
    Calculate objective value with soft time windows.

    Objective: maximize (profit - travel_cost - lateness_penalties)
    """
    total_profit = total_profit(inst, solution)
    total_travel = 0.0
    total_lateness_cost = 0.0
    total_lateness_time = 0.0

    for route in solution:
        travel, late_cost, late_time = route_cost_with_soft_tw(
            inst, route, cost_weight, lateness_weight
        )
        total_travel += travel
        total_lateness_cost += late_cost
        total_lateness_time += late_time

    net_value = (profit_weight * total_profit -
                 total_travel -
                 total_lateness_cost)

    return {
        "profit": total_profit,
        "travel_cost": total_travel,
        "lateness_cost": total_lateness_cost,
        "lateness_time": total_lateness_time,
        "net_value": net_value,
        "feasible": net_value > -1e9  # Soft feasibility
    }

## 5. Greedy insertion heuristic (initial solution)

We'll build routes one by one:

1. Start a new empty route `[0, 0]` (depot → depot).
2. While there is an uncovered customer that *can still be inserted feasibly*,
   choose the "cheapest insertion" (minimal extra distance) that keeps
   capacity and time windows feasible.
3. When no insertion is possible, **start a new vehicle** and repeat.

This is not state of the art, but it's a solid warm start for ALNS.

If the heuristic fails to insert everyone in that route, we fall back to a
single-customer route `[0, i, 0]` for that hard customer.


In [15]:
def try_insert_customer(inst: TOPTWInstance,
                         route: List[int],
                         cust_idx: int) -> Optional[Tuple[float, List[int]]]:
    """
    Try to insert `cust_idx` somewhere between route[k] -> route[k+1].
    Return (extra_cost, new_route) if feasible, else None.
    """
    best = None
    for k in range(len(route)-1):
        cand = route[:k+1] + [cust_idx] + route[k+1:]
        # capacity/time check
        if not route_capacity_ok(inst, cand):
            continue
        if not route_time_feasible(inst, cand):
            continue
        # delta distance
        a, b = route[k], route[k+1]
        old = inst.dist[a][b]
        new = inst.dist[a][cust_idx] + inst.dist[cust_idx][b]
        extra = new - old
        if best is None or extra < best[0]:
            best = (extra, cand)
    return best

def greedy_initial_solution_toptw(inst: TOPTWInstance,
                                  rng: random.Random = random.Random(0)) -> List[List[int]]:
    """
    Build initial TOPTW solution by greedily adding high-profit customers
    until no more can be feasibly added.
    """
    # Sort customers by profit density (profit per unit distance from depot)
    candidates = list(range(1, len(inst.customers)))
    candidates.sort(
        key=lambda i: inst.customers[i].profit / max(inst.dist[0][i], 1e-6),
        reverse=True
    )

    solution: List[List[int]] = []
    used_customers = set()

    # Build routes sequentially (up to max_routes)
    for _ in range(inst.max_routes):
        if not candidates:
            break

        route = [0, 0]
        route_profit = 0.0

        # Try to insert as many high-profit customers as possible
        for cidx in list(candidates):
            res = try_insert_customer(inst, route, cidx)
            if res is not None:
                extra, new_route = res
                route = new_route
                used_customers.add(cidx)
                candidates.remove(cidx)

        if route != [0, 0]:
            solution.append(route)

    return solution

## 6. Destroy & Repair Operators

**Parameter Guidance:**
- **p_remove**: 0.15-0.25 recommended (higher = more diversification)
- **Shaw removal**: Best for intensification (removes related customers)
- **Random removal**: Best for diversification

**Repair Enhancement enhanced:** The greedy repair uses profit/distance ratio for insertion ordering and tries all routes plus new route creation.

**Operator Selection Matrix:**

| Scenario | Best Destroy | p_remove | Why |
|----------|--------------|----------|-----|
| Early search (diversification) | Random | 0.25-0.30 | Explore widely |
| Late search (intensification) | Shaw | 0.15-0.20 | Refine good solutions |
| Time-critical instances | Shaw (time-aware) | 0.10-0.15 | Respect TW constraints |

**Repair Strategy**: The greedy repair sorts by `profit_weight × profit - cost_weight × distance_from_depot` to prioritize high-value insertions.

In [16]:
def remove_customers(solution: List[List[int]], to_remove: List[int]) -> List[List[int]]:
    new_sol = []
    for r in solution:
        nr = [c for c in r if (c not in to_remove) or (c==0)]
        # ensure depot endpoints remain [0,...,0]
        if nr[0] != 0:
            nr = [0] + nr
        if nr[-1] != 0:
            nr = nr + [0]
        # SKIP empty routes entirely (new)
        if len(nr) < 3:
            continue
        new_sol.append(nr)
    return new_sol

def random_removal(inst: TOPTWInstance,
                   solution: List[List[int]],
                   rng: random.Random,
                   p: float = 0.2):
    all_customers = flatten_routes(solution)
    if not all_customers:
        return solution, []
    k = max(1, int(len(all_customers)*p))
    picked = rng.sample(all_customers, k)
    new_sol = remove_customers(solution, picked)
    return new_sol, picked

def shaw_removal_time_aware(inst: TOPTWInstance,
                            solution: List[List[int]],
                            rng: random.Random,
                            p: float = 0.2,
                            compat_matrix: Optional[Dict] = None):
    """
    Shaw removal that considers both spatial and temporal relatedness.

    Uses precomputed compatibility matrix to avoid selecting customers
    that would create time window violations when reinserted together.
    """
    all_customers = flatten_routes(solution)
    if not all_customers:
        return solution, []

    # Build compatibility matrix if not provided
    if compat_matrix is None:
        compat_matrix = build_time_compatibility_matrix(inst)

    seed = rng.choice(all_customers)

    def relatedness(i, j):
        """
        Combined distance and time window relatedness.
        Lower score = more related.
        """
        # Spatial distance
        spatial = inst.dist[i][j]

        # Temporal distance (how far apart are time windows?)
        cust_i = inst.customers[i]
        cust_j = inst.customers[j]

        # Time window overlap
        tw_overlap = min(cust_i.due_time, cust_j.due_time) - \
                     max(cust_i.ready_time, cust_j.ready_time)

        # Penalize if no overlap (harder to insert in same route)
        temporal = 0.0 if tw_overlap > 0 else abs(tw_overlap)

        # Check time compatibility
        compat_bonus = 0.0
        if not compat_matrix.get((i, j), False):
            compat_bonus = 1000.0  # Heavily penalize incompatible pairs

        # Combined score (weights can be tuned)
        return spatial + 0.5 * temporal + compat_bonus

    rem_list = [seed]
    cand_pool = set(all_customers) - {seed}
    target_k = max(1, int(len(all_customers) * p))

    while len(rem_list) < target_k and cand_pool:
        # Pick most related customer
        best_j = None
        best_score = None

        for j in list(cand_pool):
            # Minimum relatedness to any already selected customer
            score = min(relatedness(i, j) for i in rem_list)

            if best_score is None or score < best_score:
                best_score = score
                best_j = j

        if best_j is None:
            break

        rem_list.append(best_j)
        cand_pool.remove(best_j)

    new_sol = remove_customers(solution, rem_list)
    return new_sol, rem_list

def repair_greedy_toptw_enhanced(inst: TOPTWInstance,
                                partial_solution: List[List[int]],
                                missing_customers: List[int],
                                rng: random.Random,
                                profit_weight: float = 1.0,
                                cost_weight: float = 0.01,
                                use_fast_check: bool = True) -> List[List[int]]:
    """
    Enhanced repair with fast insertion bounds checking.
    """
    sol = [r[:] for r in partial_solution]
    unrouted = list(missing_customers)

    # Sort by net value
    unrouted.sort(
        key=lambda i: profit_weight * inst.customers[i].profit -
                      cost_weight * inst.dist[0][i],
        reverse=True
    )

    for cidx in unrouted:
        best_score = None
        best_route = None
        best_ridx = None

        # Try inserting in existing routes
        for ridx, route in enumerate(sol):
            for insert_pos in range(len(route) - 1):
                # ENHANCEMENT: Fast bounds check before full validation
                if use_fast_check and len(route) > 2:
                    if not fast_insertion_check(inst, route, insert_pos, cidx):
                        continue  # Skip expensive full check

                # Full insertion attempt
                cand_route = route[:insert_pos+1] + [cidx] + route[insert_pos+1:]

                if not route_capacity_ok(inst, cand_route):
                    continue
                if not route_time_feasible(inst, cand_route):
                    continue

                # Calculate insertion score
                extra_cost = (inst.dist[route[insert_pos]][cidx] +
                             inst.dist[cidx][route[insert_pos+1]] -
                             inst.dist[route[insert_pos]][route[insert_pos+1]])

                profit_gain = profit_weight * inst.customers[cidx].profit
                cost_penalty = cost_weight * extra_cost
                score = profit_gain - cost_penalty

                if best_score is None or score > best_score:
                    best_score = score
                    best_route = cand_route
                    best_ridx = ridx

        if best_route is not None:
            sol[best_ridx] = best_route
        elif len(sol) < inst.max_routes:
            # Create new route
            new_route = [0, cidx, 0]
            if route_capacity_ok(inst, new_route) and route_time_feasible(inst, new_route):
                sol.append(new_route)

    return sol

## 7. ALNS Main Loop (Adaptive Large Neighborhood Search)

### **Algorithm Flow**
1. **Initialize**: Greedy solution + equal operator weights
2. **Loop** (for `iters` iterations):
   - *Select Destroy*: Weighted by past performance
   - *Destroy*: Remove ~p_remove% of customers
   - *Repair*: Reinsert using greedy heuristic with bounds checking
   - *Accept*: Metropolis criterion with temperature T
   - *Adapt*: Update operator scores every `segment_length` iterations
3. **Output**: Best feasible solution found

### **Parameter Tuning Guide**
| Parameter | Recommended Range | Effect of Increasing |
|-----------|-------------------|----------------------|
| `iters` | 300-1000 | Better solutions, slower runtime |
| `p_remove` | 0.15-0.25 | More diversification, risk of destroying good structure |
| `T0` | 5-20 | Higher acceptance of worse solutions early |
| `cooling` | 0.99-0.999 | Slower cooling = more exploration |
| `segment_length` | 20-50 | Smoother operator adaptation |
| `alpha` | 0.7-0.9 | Higher = more weight on historical performance |

**Cooling Schedule**: Exponential decay T = T₀ × cooling^iter. For 500 iterations with T₀=10, cooling=0.995 gives T_final ≈ 0.08.

### **Operator Adaptation Logic**

We maintain for each destroy operator:
- **Weight**: Starts at 1.0, evolves based on performance
- **Segment Score**: Sum of rewards earned in current segment
- **Usage Count**: How many times it was selected this segment

**Reward structure** (classical ALNS style):
-  **+5.0**  : New global best solution
-  **+2.0**  : Improved current solution
-  **+0.5**  : Accepted but worse (exploration)
-  **+0.0**  : Rejected or infeasible

Every `segment_length` iterations, weights update as:

weight_i = α × weight_i + (1-α) × (score_i / max(usage_i, 1))
Then re-normalize to probabilities and reset segment accumulators.

**Debugging Tip**: Set `segment_length=5` and `iters=50` to quickly see operator probabilities adapt in real-time during development.

In [17]:
import math

def accept_metropolis(old_cost: float, new_cost: float, T: float, rng: random.Random) -> bool:
    if new_cost <= old_cost:
        return True
    # worse -> accept with exp(-(delta)/T)
    delta = new_cost - old_cost
    prob = math.exp(-delta / max(T,1e-9))
    return (rng.random() < prob)

def weighted_choice(rng: random.Random, probs: List[float]) -> int:
    # pick index i with probability probs[i]
    s = sum(probs)
    if s <= 0:
        # fallback: uniform
        u = rng.randrange(len(probs))
        return u
    r = rng.random() * s
    acc = 0.0
    for i,p in enumerate(probs):
        acc += p
        if r <= acc:
            return i
    return len(probs)-1

def alns_enhanced(inst: TOPTWInstance,
                 iters: int = 200,
                 p_remove: float = 0.2,
                 T0: float = 10.0,
                 cooling: float = 0.995,
                 seed: int = 0,
                 alpha: float = 0.8,
                 segment_length: int = 20,
                 profit_weight: float = 1.0,
                 cost_weight: float = 0.01,
                 use_soft_tw: bool = False,           # NEW
                 lateness_weight: float = 10.0,       # NEW
                 use_time_aware_shaw: bool = True):   # NEW

    # ENHANCEMENT: Precompute compatibility matrix
    compat_matrix = build_time_compatibility_matrix(inst) if use_time_aware_shaw else None

    # --- operator pool ---
    destroy_ops = [
        ("random", random_removal),
        ("shaw", shaw_removal_time_aware if use_time_aware_shaw else shaw_removal),
    ]
    n_ops = len(destroy_ops)

    # initial equal weights / probs
    weights = [1.0 for _ in range(n_ops)]
    probs   = [1.0/n_ops for _ in range(n_ops)]

    # per-segment accumulators
    seg_score = [0.0 for _ in range(n_ops)]
    seg_used  = [0   for _ in range(n_ops)]

    # --- init solution ---
    rng = random.Random(seed)
    curr = greedy_initial_solution_toptw(inst, rng)

    # ENHANCEMENT: Choose evaluation method
    if use_soft_tw:
        eval_result = solution_value_soft_tw(inst, curr, profit_weight,
                                            cost_weight, lateness_weight)
        curr_net = eval_result["net_value"]
        curr_profit = eval_result["profit"]
        curr_cost = eval_result["travel_cost"] + eval_result["lateness_cost"]
    else:
        curr_profit = total_profit(inst, curr)
        curr_cost = total_distance(inst, curr)
        curr_net = profit_weight * curr_profit - cost_weight * curr_cost

    best = [r[:] for r in curr]
    best_profit = curr_profit
    best_cost = curr_cost
    best_net = curr_net

    hist_best_net = [best_net]
    T = T0

    # iteration log
    iter_log = []

    for it in range(iters):
        # choose destroy operator
        op_id = weighted_choice(rng, probs)
        op_name, op_fun = destroy_ops[op_id]

        # destroy
        if op_name == "shaw" and use_time_aware_shaw:
            partial, removed = op_fun(inst, curr, rng, p=p_remove,
                                     compat_matrix=compat_matrix)
        else:
            partial, removed = op_fun(inst, curr, rng, p=p_remove)

        # ENHANCEMENT: repair with fast checks
        cand = repair_greedy_toptw_enhanced(inst, partial, removed, rng,
                                           profit_weight, cost_weight,
                                           use_fast_check=True)

        accepted = False
        reward   = 0.0
        note     = ""

        # Evaluate candidate
        if use_soft_tw:
            eval_result = solution_value_soft_tw(inst, cand, profit_weight,
                                                cost_weight, lateness_weight)
            if eval_result["feasible"]:
                cand_net = eval_result["net_value"]
                cand_profit = eval_result["profit"]
                cand_cost = eval_result["travel_cost"] + eval_result["lateness_cost"]
            else:
                continue  # Skip infeasible candidates
        else:
            if solution_feasible_toptw(inst, cand):
                cand_profit = total_profit(inst, cand)
                cand_cost = total_distance(inst, cand)
                cand_net = profit_weight * cand_profit - cost_weight * cand_cost
            else:
                continue

        # Metropolis acceptance
        if accept_metropolis(-curr_net, -cand_net, T, rng):
            accepted = True

            if cand_net > best_net + 1e-9:
                reward = 5.0
                note = "new_global_best"
            elif cand_net > curr_net + 1e-9:
                reward = 2.0
                note = "improved_curr"
            else:
                reward = 0.5
                note = "accepted_worse"

            curr, curr_profit, curr_cost, curr_net = cand, cand_profit, cand_cost, cand_net

            if cand_net > best_net + 1e-9:
                best, best_profit, best_cost, best_net = [r[:] for r in cand], cand_profit, cand_cost, cand_net

        # bookkeeping
        seg_score[op_id] += reward
        seg_used[op_id]  += 1

        # log iteration
        iter_log.append({
            "iter": it,
            "T": T,
            "op_id": op_id,
            "op_name": op_name,
            "accepted": accepted,
            "reward": reward,
            "note": note,
            "curr_net": curr_net,
            "best_net": best_net,
            "curr_profit": curr_profit,
            "best_profit": best_profit,
            "curr_cost": curr_cost,
            "best_cost": best_cost,
            "probs_snapshot": probs[:],
        })

        # cooling
        hist_best_net.append(best_net)
        T *= cooling

        # segment update
        if (it+1) % segment_length == 0:
            for j in range(n_ops):
                if seg_used[j] > 0:
                    avg_score = seg_score[j] / seg_used[j]
                else:
                    avg_score = 0.0
                weights[j] = alpha * weights[j] + (1.0-alpha) * avg_score

            s = sum(weights)
            if s <= 0:
                probs = [1.0/n_ops for _ in range(n_ops)]
            else:
                probs = [w/s for w in weights]

            seg_score = [0.0 for _ in range(n_ops)]
            seg_used  = [0   for _ in range(n_ops)]

    return {
        "best_solution": best,
        "best_profit": best_profit,
        "best_cost": best_cost,
        "best_net": best_net,
        "final_probs": probs,
        "op_usage": {name: seg_used[i] for i, (name, _) in enumerate(destroy_ops)},
        "iter_log": iter_log,
        "history": hist_best_net,
    }

## **8. Troubleshooting Guide**

**Problem: All solutions infeasible**
- Check that depot time window `[tw_open, tw_close]` spans all customers
- Verify `max_routes` in JSON matches `inst.max_routes`
- Temporarily disable fast insertion checks

**Problem: Operator probabilities don't adapt**
- Reduce `segment_length` to 10-15
- Increase `alpha` to 0.9 (more memory)
- Check that rewards are being assigned (print `seg_score`)

**Problem: Poor solution quality**
- Increase `iters` to 500-1000
- Reduce `cost_weight` (e.g., 0.05 → 0.01) to prioritize profit
- Try `use_time_aware_shaw=True` for tight TW instances

**Problem: Runtime too slow**
- Enable `use_fast_check=True`
- Reduce `iters` or `p_remove`
- Precompute distance matrix once and reuse

In [18]:
# Run the comprehensive TOPTW experiment from LOCAL folder

# =====================================================================
# CONFIGURATION: Set your local folder paths here
# =====================================================================
DATA_FOLDER = './benchmarks_cap_json'      # Folder containing TOPTW JSON files
RESULTS_FOLDER = './toptw_results'         # Folder for saving results

# =====================================================================
# Setup and verification
# =====================================================================
import os

# Create results folder if it doesn't exist
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Verify data folder exists and has files
if not os.path.isdir(DATA_FOLDER):
    print(f"❌ ERROR: Data folder '{DATA_FOLDER}' does not exist!")
    print(f"Current working directory: {os.getcwd()}")
    print("\nPlease either:")
    print("1. Create the folder and add your JSON files, or")
    print("2. Update DATA_FOLDER path in the code")
    raise FileNotFoundError(f"Data folder not found: {DATA_FOLDER}")

# Count JSON files
json_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(('.json', '.JSON'))]
if len(json_files) == 0:
    print(f"⚠️  WARNING: No JSON files found in '{DATA_FOLDER}'")
    print("Please add your TOPTW instance files (*.json)")
else:
    print(f"✅ Found {len(json_files)} JSON files in '{DATA_FOLDER}'")

# =====================================================================
# Run experiment
# =====================================================================
print("\n" + "=" * 60)
print("STARTING TOPTW EXPERIMENT FROM LOCAL FOLDER")
print("=" * 60)
print(f"Data source: {os.path.abspath(DATA_FOLDER)}")
print(f"Results destination: {os.path.abspath(RESULTS_FOLDER)}")
print("=" * 60)

df_results, df_summary = run_folder_experiment_toptw(
    DATA_FOLDER,
    repeats=5,          # Number of runs per instance
    iters=500,          # ALNS iterations
    base_seed=1000,
    select="first_n",   # Process first n instances
    n=2,                # Number of instances to process
    profit_weight=1.0,  # Profit importance
    cost_weight=0.1     # Cost penalty factor
)

# =====================================================================
# Display results
# =====================================================================
print("\n" + "=" * 60)
print("EXPERIMENT COMPLETED")
print("=" * 60)

print("\n📊 DETAILED RESULTS (First 10 rows):")
display(df_results.head(10))

print("\n📈 SUMMARY STATISTICS:")
display(df_summary)

# Performance analysis
if not df_summary.empty:
    print("\n🔍 PERFORMANCE ANALYSIS:")
    print(f"Total instances tested: {len(df_summary)}")
    print(f"Average feasibility rate: {df_summary['feas_rate'].mean():.2%}")
    print(f"Average ALNS time: {df_summary['mean_time'].mean():.2f} seconds")
    print(f"Average net value: {df_summary['mean_net_value'].mean():.2f}")
    print(f"Best net value found: {df_summary['max_net_value'].max():.2f}")
    print(f"Improvement over baseline: {df_summary['improvement_over_baseline'].mean():.2%}")

# =====================================================================
# Save results locally
# =====================================================================
try:
    import time
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    results_file = os.path.join(RESULTS_FOLDER, f'toptw_results_{timestamp}.csv')
    summary_file = os.path.join(RESULTS_FOLDER, f'toptw_summary_{timestamp}.csv')
    
    df_results.to_csv(results_file, index=False)
    df_summary.to_csv(summary_file, index=False)
    
    print(f"\n💾 Results saved to local folder:")
    print(f"   - {results_file}")
    print(f"   - {summary_file}")
    
    # Show all files in results folder
    print(f"\n📁 Files in {RESULTS_FOLDER}:")
    for f in sorted(os.listdir(RESULTS_FOLDER)):
        print(f"   - {f}")
        
except Exception as e:
    print(f"\n❌ Could not save results: {e}")
    import traceback
    traceback.print_exc()

❌ ERROR: Data folder './benchmarks_cap_json' does not exist!
Current working directory: /home/thijs-alens/Documents/manama/applied-AI_academic-perspectives/combinatorial-optimization/project/Amine

Please either:
1. Create the folder and add your JSON files, or
2. Update DATA_FOLDER path in the code


FileNotFoundError: Data folder not found: ./benchmarks_cap_json

### **How to Interpret Results**

- **Improvement %**: Values &gt;0% mean ALNS beat the greedy baseline. 40-60% is typical.
- **Feasibility Rate**: Should be 100%. If not, check time windows or capacity constraints.
- **Operator Usage**: In `op_usage`, if one operator dominates (&gt;70%), increase exploration (`p_remove`, `T0`).
- **Net Value**: The true objective. Compare across different `λ` (cost_weight) values to find your optimal tradeoff.

### **Next Steps**
1. **Parameter Sweep**: Run with `cost_weight = [0.01, 0.05, 0.1, 0.2]` to plot the Pareto frontier.
2. **Larger Instances**: Test on 100+ customer instances by increasing `iters` and `max_routes`.
3. **Custom Operators**: Add specialized destroy (e.g., worst-profit removal) or repair (e.g., regret insertion).